In [2]:
import numpy as np
from math import pi, sqrt 

In [3]:
# check if the qbit is valid :Todo
q0 = np.array([0,1])
q1 = np.array([1/sqrt(2),1/sqrt(2)])
p1 = np.abs(q1)**2
print(int(round(p1.sum(),5)))

1


In [4]:
combined_state = np.kron(q0,q1)
print(combined_state)

[0.         0.         0.70710678 0.70710678]


In [5]:
X = np.array([
    [0,1],
    [1,0]
])

q0 = np.dot(X,q0)
print("Final state:\t", q0)

Final state:	 [1 0]


In [6]:
psi = [1,0,0,0,0,0,0,0]
print("Initial state:", psi)

X = np.array([
    [0,1],
    [1,0]
])

I = np.identity(2)

O = np.kron(np.kron(I, I), X)

print("\nOperator:\n\n", O, "\n")

psi = np.dot(psi, O)
print("Final state:", psi)

Initial state: [1, 0, 0, 0, 0, 0, 0, 0]

Operator:

 [[0. 1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 1. 0.]] 

Final state: [0. 1. 0. 0. 0. 0. 0. 0.]


In [7]:
P0x0 = np.array ([
    [1,0],
    [0,0]
])

P1x1 = np.array([
[0, 0],
[0, 1]
])

O = np.kron(np.kron(P0x0, I), I) + np.kron(np.kron(P1x1, I), X)

print(O)

[[1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 1. 0.]]


In [8]:
from numpy.random import choice
qbit = {
    0: 0,
    1: 0
}
for i in range(100):
    draw = choice(2,1,p=[0.1,0.9])
    if draw[0] == 0:
        qbit[0]+=1
    else:
        qbit[1]+=1
print(qbit)

{0: 10, 1: 90}


In [19]:
def get_ground_state(num_qubits):
    # return vector of size 2**num_qubits with all zeroes except first element which is 1
    if(num_qubits <=0):
        return "Please enter valid number greater than 0"
    q = [1,0]
    for i in range(num_qubits-1):
        q = np.kron(q,[1,0])
    return q

def get_operator(total_qubits, gate_unitary, target_qubits):
    # return unitary operator of size 2**n x 2**n for given gate and target qubits
    switcher = {
        "x": x_gate,
        "h": hadamard_gate
    }
    gate_func = switcher.get(gate_unitary, lambda: "Invalid gate name")
    return func()

def hadamard_gate():
    return "hadamard"

def x_gate():
    return "x_gate"

print(get_operator(2,"h",2))

def run_program(initial_state, program):
    # read program, and for each gate:
    #   - calculate matrix operator
    #   - multiply state with operator
    # return final state
    return
print(get_ground_state(-2))

NameError: name 'func' is not defined